In [ ]:
!pip3 install catboost

In [ ]:
import pandas as pd
import numpy as np

csv_path = "trades-df-2021_11_1_0_0_0-2021_12_21_0_0_0.csv"
df = pd.read_csv(csv_path)

In [ ]:
x_columns = list(filter(lambda column: not 'punch' in column, df.columns))
y_columns = list(filter(lambda column: 'punch' in column, df.columns))
print(x_columns)
print(y_columns)

['trades-BUY-6-sec', 'trades-SELL-6-sec', 'trades-BUY-12-sec', 'trades-SELL-12-sec', 'trades-BUY-18-sec', 'trades-SELL-18-sec', 'trades-BUY-24-sec', 'trades-SELL-24-sec', 'trades-BUY-30-sec', 'trades-SELL-30-sec', 'trades-BUY-36-sec', 'trades-SELL-36-sec', 'trades-BUY-42-sec', 'trades-SELL-42-sec', 'trades-BUY-48-sec', 'trades-SELL-48-sec', 'trades-BUY-54-sec', 'trades-SELL-54-sec', 'trades-BUY-60-sec', 'trades-SELL-60-sec']
['punch-BUY-6-sec', 'punch-SELL-6-sec', 'punch-BUY-12-sec', 'punch-SELL-12-sec', 'punch-BUY-18-sec', 'punch-SELL-18-sec', 'punch-BUY-24-sec', 'punch-SELL-24-sec', 'punch-BUY-30-sec', 'punch-SELL-30-sec', 'punch-BUY-36-sec', 'punch-SELL-36-sec', 'punch-BUY-42-sec', 'punch-SELL-42-sec', 'punch-BUY-48-sec', 'punch-SELL-48-sec', 'punch-BUY-54-sec', 'punch-SELL-54-sec', 'punch-BUY-60-sec', 'punch-SELL-60-sec']


In [ ]:
X = df[x_columns]
secs = list(reversed(sorted(list(set(map(lambda x : int(''.join(x)), [[s for s in column if s.isdigit()] for column in x_columns]))))))

for side in ['BUY', 'SELL']:
  for s in secs[:-1]:
    X[f'trades-{side}-{str(s)}-sec'] -= X[f'trades-{side}-{str(s - (secs[0] - secs[1]))}-sec']
X

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,trades-BUY-6-sec,trades-SELL-6-sec,trades-BUY-12-sec,trades-SELL-12-sec,trades-BUY-18-sec,trades-SELL-18-sec,trades-BUY-24-sec,trades-SELL-24-sec,trades-BUY-30-sec,trades-SELL-30-sec,trades-BUY-36-sec,trades-SELL-36-sec,trades-BUY-42-sec,trades-SELL-42-sec,trades-BUY-48-sec,trades-SELL-48-sec,trades-BUY-54-sec,trades-SELL-54-sec,trades-BUY-60-sec,trades-SELL-60-sec
0,0.100,0.200,0.400,0.300,10.255,0.200,0.100,0.500,0.300,0.200,0.216,2.700,0.200,0.200,0.200,2.066,0.000,2.600,0.500,9.981
1,0.200,0.400,0.300,3.684,0.200,0.366,0.300,0.400,0.250,0.000,0.200,0.200,0.749,0.400,0.100,0.000,0.200,33.531,0.800,0.100
2,0.200,2.434,49.298,139.475,0.300,4.486,0.100,5.982,0.200,5.806,0.400,7.355,0.246,9.057,0.100,0.300,0.220,1.000,0.200,0.300
3,2.513,9.616,4.975,5.386,36.317,35.898,0.200,8.477,0.000,0.300,0.400,5.400,9.516,0.150,0.000,0.200,0.000,2.181,0.200,24.480
4,5.152,0.200,0.100,6.006,0.100,9.416,6.790,0.400,9.316,0.100,1.167,0.100,0.500,9.616,2.439,0.200,161.648,0.200,6.105,9.969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39941,5.369,10.249,0.000,0.000,9.999,10.499,0.000,0.000,309.999,9.999,0.000,3.000,9.999,9.215,1.038,0.000,9.215,2.645,0.000,0.000
39942,7.765,20.339,0.000,0.414,9.999,9.999,0.000,0.000,0.000,26.878,1.914,0.000,0.000,25.819,0.035,0.000,0.000,101.502,0.000,5.583
39943,9.999,1.530,31.055,0.000,9.999,9.999,9.999,0.067,0.000,30.003,0.000,584.297,0.000,25.896,0.000,0.000,2.938,17.938,0.000,32.400
39944,0.802,1.381,0.861,0.000,5.582,0.000,10.000,3.005,10.000,86.202,0.000,0.000,90.906,155.992,19.788,48.927,1.134,0.000,197.118,0.000


In [ ]:
commision = 0.0002
punch = commision * 6

y = pd.DataFrame(
    df[y_columns].apply(
        lambda row : 1 if max(list(row.to_numpy()), key=abs) > punch else -1 if max(list(row.to_numpy()), key=abs) < -punch else 0,
        axis = 1
    )
)
y = y.rename({0:'target'}, axis='columns')
print(y)
print(y.value_counts())

       target
0           0
1           0
2           1
3           1
4           0
...       ...
39941       0
39942       0
39943       0
39944       0
39945       0

[39946 rows x 1 columns]
target
 0        32768
-1         3938
 1         3240
dtype: int64


In [ ]:
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle = True, random_state=12)

X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, train_size=0.75, shuffle = True, random_state=6)

In [ ]:
params = {
    'iterations': 500,
    'learning_rate': 0.1,
    'depth': 5,
    'custom_loss': [metrics.Accuracy()],
    'eval_metric': metrics.Accuracy(),
    'random_seed': 42,
    'logging_level': 'Silent',
    'loss_function': 'MultiClass',
}
train_pool = Pool(X_train, y_train)
validate_pool = Pool(X_validation, y_validation)

model = CatBoostClassifier(**params)
model.fit(
    train_pool,
    eval_set=validate_pool,
    plot=True,
)
print('Model validation accuracy: {:.4}'.format(
    accuracy_score(y_test, model.predict(X_test))
))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Model validation accuracy: 0.8249


In [ ]:
cv_params = model.get_params()
cv_params.update({
    'loss_function': metrics.MultiClass()
})
cv_data = cv(
    Pool(X, y),
    cv_params,
    plot=True
)
print('Best validation accuracy score: {:.2f}±{:.2f} on step {}'.format(
    np.max(cv_data['test-Accuracy-mean']),
    cv_data['test-Accuracy-std'][np.argmax(cv_data['test-Accuracy-mean'])],
    np.argmax(cv_data['test-Accuracy-mean'])
))
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
predictions = model.predict(X_test)
predictions_probs = model.predict_proba(X_test)
print(predictions[10:20])
print(predictions_probs[10:20])
print(y_test[10:20])

[[ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 1]
 [ 1]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [-1]
 [ 0]
 [ 0]
 [ 0]]
[[0.12181386 0.83753758 0.04064857]
 [0.05931181 0.89331009 0.0473781 ]
 [0.13522795 0.79207171 0.07270034]
 [0.12990134 0.70816834 0.16193032]
 [0.41101146 0.17227787 0.41671067]
 [0.21829169 0.38679455 0.39491377]
 [0.06966344 0.85991401 0.07042255]
 [0.14168047 0.75161677 0.10670276]
 [0.06364222 0.8810044  0.05535338]
 [0.05913479 0.87988735 0.06097786]
 [0.20376204 0.67043737 0.12580059]
 [0.21764688 0.55828943 0.22406369]
 [0.10566177 0.75927453 0.1350637 ]
 [0.06402465 0.88299595 0.0529794 ]
 [0.11988235 0.81284849 0.06726916]
 [0.20642697 0.67586145 0.11771158]
 [0.35424278 0.29573972 0.3500175 ]
 [0.1076348  0.81630981 0.07605539]
 [0.13319562 0.79645986 0.07034452]
 [0.16396788 0.58336611 0.25266601]]
       target
902         0
3804        0
33418       0
35239       0
18256       1
26745       1
7390        0
35376       0
23093       0
16538       0
3477

In [ ]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))


trades-SELL-6-sec: 7.7659552186603396
trades-SELL-18-sec: 6.223632899002721
trades-SELL-60-sec: 5.915927340178629
trades-BUY-6-sec: 5.6560416519685335
trades-BUY-24-sec: 5.637289386815484
trades-SELL-30-sec: 5.388997726562737
trades-SELL-54-sec: 5.250202741138116
trades-BUY-60-sec: 5.103974093412608
trades-SELL-12-sec: 5.095218006563516
trades-BUY-18-sec: 5.086850941345767
trades-SELL-24-sec: 5.048694966896342
trades-BUY-36-sec: 4.895561730137529
trades-BUY-30-sec: 4.790538384504451
trades-BUY-48-sec: 4.602331031844483
trades-BUY-42-sec: 4.059449891446964
trades-BUY-54-sec: 3.974965475726563
trades-BUY-12-sec: 3.969556583428052
trades-SELL-36-sec: 3.9451593384617762
trades-SELL-48-sec: 3.881826755018264
trades-SELL-42-sec: 3.7078258368871397
